# SALESFORCE INTEGRATION

## OAuth 2.0 JWT Bearer Flow for Server-to-Server Integration

[Link Salesforce documentation](https://help.salesforce.com/articleView?id=sf.remoteaccess_oauth_jwt_flow.htm&type=5)

Sometimes you want to authorize servers to access data without interactively logging in each time the servers exchange information. For these cases, you can use the OAuth 2.0 JSON Web Token (JWT) bearer flow. This flow uses a certificate to sign the JWT request and doesn’t require explicit user interaction. However, this flow does require prior approval of the client app.

Settings:


* **SF_SECRET_ISS** -The issuer must contain the OAuth client_id or the connected app for which you registered the certificate.

* **SF_SECRET_AUD** - The audience identifies the authorization server as an intended audience. The authorization server must verify that it’s an intended audience for the token. Use the authorization server’s URL for the audience value: https://login.salesforce.com, https://test.salesforce.com, or https://site.force.com/customers if implementing for an Experience Cloud site.

* **SF_SECRET_AUD** - The subject must contain the username of the user if implementing for an Experience Cloud site. For backward compatibility, you can use principal (prn) instead of subject (sub). If both are specified, prn is used.

* **SF_SECRET_SUB** - The validity must be the expiration time of the assertion within 3 minutes, expressed as the number of seconds from 1970-01-01T0:0:0Z measured in UTC.


In [ ]:
from __future__ import print_function
from datetime import datetime, timedelta
from base64 import (
    b64encode,
    b64decode,
)

from Crypto.Hash import SHA256
from Crypto.Signature import PKCS1_v1_5
from Crypto.PublicKey import RSA

import requests
import json
import uuid
import sys, base64, textwrap
import jks
import time
import requests

In [ ]:
url_salesforce = ""

SF_JKS_FILE_PATH="../input/sf.jks"
SF_JKS_FILE_PHASE=""

SF_SECRET_ISS = ""
SF_SECRET_AUD = ""
SF_SECRET_SUB = ""
SF_SECRET_EXP = str(int((datetime.now() + timedelta(hours=1)).timestamp()*1000))


### Get Auth Token

In [ ]:
def print_pem(der_bytes, type):
    print("-----BEGIN %s-----" % type)
    print("\r\n".join(textwrap.wrap(base64.urlsafe_b64encode(der_bytes).decode('ascii'), 64)))
    print("-----END %s-----" % type)

def converter_base64(message):
    message_bytes = message.encode('ascii')
    base64_bytes = base64.urlsafe_b64encode(message_bytes)
    base64_message = base64_bytes.decode('ascii')
    return base64_message
    
    
ks = jks.KeyStore.load(SF_JKS_FILE_PATH, SF_JKS_FILE_PHASE)

for alias, pk in ks.private_keys.items():
    print("Private key: %s" % pk.alias)
    if pk.algorithm_oid == jks.util.RSA_ENCRYPTION_OID:
        rsa_private_key_base64 = base64.urlsafe_b64encode(pk.pkey).decode('ascii')
        rsa_private_key = pk.pkey
    else:
        print_pem(pk.pkey_pkcs8, "PRIVATE KEY")

sf_header = converter_base64('{"alg":"RS256"}')
sf_payload_string= '{"iss": "SF_SECRET_ISS", "sub": "SF_SECRET_SUB", "aud": "SF_SECRET_AUD", "exp": "SF_SECRET_EXP"}'
sf_payload_string = sf_payload_string.replace("SF_SECRET_ISS", SF_SECRET_ISS)
sf_payload_string = sf_payload_string.replace("SF_SECRET_SUB", SF_SECRET_SUB)
sf_payload_string = sf_payload_string.replace("SF_SECRET_AUD", SF_SECRET_AUD)
sf_payload_string = sf_payload_string.replace("SF_SECRET_EXP", SF_SECRET_EXP)

print(sf_payload_string)
print('---------')
sf_payload = converter_base64(sf_payload_string)

token_sf = "{}.{}".format(sf_header, sf_payload)
print(token_sf)
print(sf_payload_string)

# Load private key and sign messagez
digest = SHA256.new()
digest.update(token_sf.encode('utf-8'))
private_key = RSA.importKey(rsa_private_key)
signer = PKCS1_v1_5.new(private_key)
sig = signer.sign(digest)

token_sf_sign = "{}.{}".format(token_sf, base64.urlsafe_b64encode(sig).decode('ascii'))

url = url_salesforce + "/services/oauth2/token"

payload='grant_type=urn%3Aietf%3Aparams%3Aoauth%3Agrant-type%3Ajwt-bearer&assertion={}'.format(token_sf_sign)
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',

}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)
sf_access_token = 'Bearer '+response.json()['access_token']
print(sf_access_token)  


In [ ]:
print(sf_access_token)

### Invoke API

In [ ]:
url = url_salesforce + "/services/data/v50.0"

payload={}
headers = {
  'Content-Type': 'application/json',
  'charset': 'UTF-8',
  'Accept': 'application/json',
  'Authorization': sf_access_token,
  
}

response = requests.request("GET", url, headers=headers, data=payload)

print(json.dumps(response.json(), indent=2))